# 인공지능과 기계학습 9주차 과제
IT공학전공 2012375 
- WINE 데이터셋에서 타겟을 class 0 vs. class 1 이진 분류용 레이블로 변환해 SVM 기반 이진 분류를 진행함

In [1]:
from sklearn.datasets import load_wine
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

## 1. 데이터 로드 & 특징 및 타겟 정의

In [2]:
# 1. 데이터 로드 & 특징 및 타겟 정의
wine = load_wine()
X_all = pd.DataFrame(wine.data, columns=wine.feature_names)
y_all = wine.target

In [3]:
# 1. 타겟을 이진 분류용 레이블로 변환

# class 0과 class 1만 선택
mask = (y_all == 0) | (y_all == 1)
X = X_all[mask]
y = y_all[mask]

# 타겟 이진화: class 0은 0, class 1은 1
y = np.where(y == 0, 0, 1)

# 데이터 특성 파악
print("기본 정보:")
print(X.info())
print("\n 통계 요약:")
print(X.describe())
print("\n 타겟 분포:")
print(pd.Series(y).value_counts())

기본 정보:
<class 'pandas.core.frame.DataFrame'>
Index: 130 entries, 0 to 129
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       130 non-null    float64
 1   malic_acid                    130 non-null    float64
 2   ash                           130 non-null    float64
 3   alcalinity_of_ash             130 non-null    float64
 4   magnesium                     130 non-null    float64
 5   total_phenols                 130 non-null    float64
 6   flavanoids                    130 non-null    float64
 7   nonflavanoid_phenols          130 non-null    float64
 8   proanthocyanins               130 non-null    float64
 9   color_intensity               130 non-null    float64
 10  hue                           130 non-null    float64
 11  od280/od315_of_diluted_wines  130 non-null    float64
 12  proline                       130 non-null    float64
dtypes: 

## 2. 데이터 분할

In [4]:
# 2. 데이터 분할 (7:3 비율, random_state = 42 정의)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

## 3. 파이프라인 구성

In [5]:
pipeline = Pipeline([
    ('scaler', StandardScaler()), # 정규화
    ('svc', SVC()) # SVM 모델 정의
])

## 4. 하이퍼파라미터 튜닝

In [6]:
# 4. 하이퍼파라미터 튜닝 (Grid Search)

param_grid = [
    {
        'svc__kernel': ['linear'],
        'svc__C': [0.01, 0.1, 1, 10, 100]
    },
    {
        'svc__kernel': ['rbf'],
        'svc__C': [0.1, 1, 10, 100],
        'svc__gamma': [0.01, 0.1, 1, 10]
    }
]

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

print('최적 하이퍼파라미터:', grid_search.best_params_)

최적 하이퍼파라미터: {'svc__C': 1, 'svc__kernel': 'linear'}


## 5. 최종 성능 평가

In [7]:
# 5. 최종 성능 평가

# 최적 하이퍼파라미터 출력
print("최적 하이퍼파라미터 셋 확인:")
print(grid_search.best_params_)

# 테스트 데이터 예측 및 평가 
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("\n테스트 정확도:", accuracy_score(y_test, y_pred))
print("\n분류 성능 리포트:\n")
print(classification_report(y_test, y_pred))

최적 하이퍼파라미터 셋 확인:
{'svc__C': 1, 'svc__kernel': 'linear'}

테스트 정확도: 1.0

분류 성능 리포트:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        19

    accuracy                           1.00        39
   macro avg       1.00      1.00      1.00        39
weighted avg       1.00      1.00      1.00        39

